In [14]:
import csv
from io import StringIO
from pprint import pformat, pprint

import pandas as pd
from pydfs_lineup_optimizer import get_optimizer, Player, Site, Sport

# The Player object

## Creating Player instances

In [2]:
p = Player?

Init signature:
Player(
    player_id: str,
    first_name: str,
    last_name: str,
    positions: List[str],
    team: str,
    salary: float,
    fppg: float,
    is_injured: bool = False,
    max_exposure: Optional[float] = None,
    min_exposure: Optional[float] = None,
    projected_ownership: Optional[float] = None,
    game_info: Optional[pydfs_lineup_optimizer.player.GameInfo] = None,
    roster_order: Optional[int] = None,
    min_deviation: Optional[float] = None,
    max_deviation: Optional[float] = None,
    is_confirmed_starter: Optional[bool] = None,
    fppg_floor: Optional[float] = None,
    fppg_ceil: Optional[float] = None,
    original_positions: Optional[List[str]] = None,
)
Docstring:      <no docstring>
File:           ~/miniconda3/envs/lineup-optimization-cookbook/lib/python3.9/site-packages/pydfs_lineup_optimizer/player.py
Type:           type
Subclasses:     


In [5]:
# At a minimum, need to pass these seven parameters
# 'player_id', 
# 'first_name', 
# 'last_name', 
# 'positions', 
# 'team', 
# 'salary', 
# 'fppg'
p = Player()

TypeError: __init__() missing 7 required positional arguments: 'player_id', 'first_name', 'last_name', 'positions', 'team', 'salary', and 'fppg'

In [6]:
# instantiate a player object using the bare necessities
p = Player(
    player_id=1,
    first_name='Patrick',
    last_name='Mahomes',
    positions=['QB'],
    team='KC',
    salary=7800,
    fppg=25.5
)

## Loading players from csv file

In [34]:
data = StringIO("""player_id,first_name,last_name,positions,team,salary,fppg
1,Patrick,Mahomes,QB,KC,7900,25.5
2,Tom,Brady,QB,TB,7200,22.3
3,Ronald,Jones,RB,TB,4400,8.9
4,Leonard,Fournette,RB,TB,5700,12.8
5,Darrel,Williams,RB,KC,4900,9.3
6,Clyde,Edwards-Helaire,RB,KC,5700,11.6
7,LeVeon,Bell,RB,KC,3900,4.2
8,Keyshawn,Vaughn,RB,TB,3700,4.0
9,Chris,Godwin,WR,TB,7100,18.2
10,Tyreek,Hill,WR,KC,7700,19.4
11,Sammy,Watkins,WR,KC,4700,11.1
12,Antonio,Brown,WR,TB,5200,12.2
13,Mike,Evans,WR,TB,6600,15.8
14,Scotty,Miller,WR,TB,4200,8.8
15,Mecole,Hardman,WR,KC,4100,8.7
16,Travis,Kelce,TE,KC,8200,20.4
17,Rob,Gronkowski,TE,TB,4400,9.3
18,Cameron,Brate,TE,TB,4600,10.1
19,Kansas City,Defense,DST,KC,3300,5.2
20,Tampa,Defense,DST,TB,3100,5.1
""")

In [37]:
players = []
for row in csv.DictReader(data):
    players.append(Player(**row))

In [38]:
players

[Patrick Mahomes Q/B (KC),
 Tom Brady Q/B (TB),
 Ronald Jones R/B (TB),
 Leonard Fournette R/B (TB),
 Darrel Williams R/B (KC),
 Clyde Edwards-Helaire R/B (KC),
 LeVeon Bell R/B (KC),
 Keyshawn Vaughn R/B (TB),
 Chris Godwin W/R (TB),
 Tyreek Hill W/R (KC),
 Sammy Watkins W/R (KC),
 Antonio Brown W/R (TB),
 Mike Evans W/R (TB),
 Scotty Miller W/R (TB),
 Mecole Hardman W/R (KC),
 Travis Kelce T/E (KC),
 Rob Gronkowski T/E (TB),
 Cameron Brate T/E (TB),
 Kansas City Defense D/S/T (KC),
 Tampa Defense D/S/T (TB)]

## Loading players from pandas dataframe

In [39]:
data = StringIO("""player_id,first_name,last_name,positions,team,salary,fppg,projected_ownership
1,Patrick,Mahomes,QB,KC,7900,25.5,32
2,Tom,Brady,QB,TB,7200,22.3,28
3,Ronald,Jones,RB,TB,4400,8.9,11
4,Leonard,Fournette,RB,TB,5700,12.8,17
5,Darrel,Williams,RB,KC,4900,9.3,14
6,Clyde,Edwards-Helaire,RB,KC,5700,11.6,19
7,LeVeon,Bell,RB,KC,3900,4.2,8
8,Keyshawn,Vaughn,RB,TB,3700,4.0,2
9,Chris,Godwin,WR,TB,7100,18.2,41
10,Tyreek,Hill,WR,KC,7700,19.4,39
11,Sammy,Watkins,WR,KC,4700,11.1,22
12,Antonio,Brown,WR,TB,5200,12.2,18
13,Mike,Evans,WR,TB,6600,15.8,36
14,Scotty,Miller,WR,TB,4200,8.8,12
15,Mecole,Hardman,WR,KC,4100,8.7,7
16,Travis,Kelce,TE,KC,8200,20.4,59
17,Rob,Gronkowski,TE,TB,4400,9.3,22
18,Cameron,Brate,TE,TB,4600,10.1,19
19,Kansas City,Defense,DST,KC,3300,5.2,60
20,Tampa,Defense,DST,TB,3100,5.1,40
""")

In [40]:
# assume you have a dataframe with the columns above
df = pd.read_csv(data)

In [41]:
def row_to_player(row):
    """Converts dataframe row to Player object
    
    Args:
        row (namedtuple): dataframe row
    
    Returns:
        Player
    """
    valid_kwargs = {
        'is_injured', 'max_exposure', 'min_exposure', 
        'projected_ownership', 'game_info', 'roster_order', 
        'min_deviation', 'max_deviation', 'is_confirmed_starter', 
        'fppg_floor', 'fppg_ceil'
    }

    kwargs = {}
    for name, value in row._asdict().items():
        if name in valid_kwargs:
            kwargs[name] = value    
    
    return Player (
            player_id=getattr(row, 'id') if hasattr(row, 'id') else row.first_name + ' ' + row.last_name, 
            first_name=row.first_name,
            last_name=row.last_name,
            positions=[row.positions],
            team=row.team,
            salary=row.salary,
            fppg=row.fppg,
            **kwargs
    )

players = [row_to_player(row) for row in df.itertuples(index=False)]

In [42]:
players

[Patrick Mahomes QB (KC),
 Tom Brady QB (TB),
 Ronald Jones RB (TB),
 Leonard Fournette RB (TB),
 Darrel Williams RB (KC),
 Clyde Edwards-Helaire RB (KC),
 LeVeon Bell RB (KC),
 Keyshawn Vaughn RB (TB),
 Chris Godwin WR (TB),
 Tyreek Hill WR (KC),
 Sammy Watkins WR (KC),
 Antonio Brown WR (TB),
 Mike Evans WR (TB),
 Scotty Miller WR (TB),
 Mecole Hardman WR (KC),
 Travis Kelce TE (KC),
 Rob Gronkowski TE (TB),
 Cameron Brate TE (TB),
 Kansas City Defense DST (KC),
 Tampa Defense DST (TB)]

## Inspecting Player objects

In [4]:
# when we print Player, only get limited information
p = Player(
    player_id=1,
    first_name='Patrick',
    last_name='Mahomes',
    positions=['QB'],
    team='KC',
    salary=7800,
    fppg=25.5
)

print(p)

Patrick Mahomes QB (KC)


In [5]:
# controlled by object's __repr__ method
# here it is only the full_name, positions, team
Player.__repr__??

Signature: Player.__repr__(self)
Docstring: Return repr(self).
Source:   
    def __repr__(self):
        return '%s %s (%s)' % (self.full_name, '/'.join(self.positions), self.team)
File:      ~/miniconda3/envs/lineup-optimization-cookbook/lib/python3.9/site-packages/pydfs_lineup_optimizer/player.py
Type:      function


In [9]:
# if we want more information, can either subclass player
# or can also have a function to print Player objects
class PrettyPlayer(Player):
    def __repr__(self):
        return pformat(vars(p), indent=2)

def print_player(p):
    pprint(vars(p), indent=2)

pp = PrettyPlayer(
    player_id=1,
    first_name='Patrick',
    last_name='Mahomes',
    positions=['QB'],
    team='KC',
    salary=7800,
    fppg=25.5
)

print('Using print_player function')
print_player(p)
print()
print('Using subclassed object')
print(pp)

Using print_player function
{ '_max_deviation': None,
  '_max_exposure': None,
  '_min_deviation': None,
  '_min_exposure': None,
  '_original_positions': None,
  '_projected_ownership': None,
  'first_name': 'Patrick',
  'fppg': 25.5,
  'fppg_ceil': None,
  'fppg_floor': None,
  'game_info': None,
  'id': 1,
  'is_confirmed_starter': None,
  'is_injured': False,
  'last_name': 'Mahomes',
  'positions': ['QB'],
  'roster_order': None,
  'salary': 7800,
  'team': 'KC'}

Using subclassed object
{ '_max_deviation': None,
  '_max_exposure': None,
  '_min_deviation': None,
  '_min_exposure': None,
  '_original_positions': None,
  '_projected_ownership': None,
  'first_name': 'Patrick',
  'fppg': 25.5,
  'fppg_ceil': None,
  'fppg_floor': None,
  'game_info': None,
  'id': 1,
  'is_confirmed_starter': None,
  'is_injured': False,
  'last_name': 'Mahomes',
  'positions': ['QB'],
  'roster_order': None,
  'salary': 7800,
  'team': 'KC'}


# Loading Players

In [43]:
o = get_optimizer(site=Site.DRAFTKINGS, sport=Sport.FOOTBALL)

In [44]:
data = StringIO("""player_id,first_name,last_name,positions,team,salary,fppg
1,Patrick,Mahomes,QB,KC,7900,25.5
2,Tom,Brady,QB,TB,7200,22.3
3,Ronald,Jones,RB,TB,4400,8.9
4,Leonard,Fournette,RB,TB,5700,12.8
5,Darrel,Williams,RB,KC,4900,9.3
6,Clyde,Edwards-Helaire,RB,KC,5700,11.6
7,LeVeon,Bell,RB,KC,3900,4.2
8,Keyshawn,Vaughn,RB,TB,3700,4.0
9,Chris,Godwin,WR,TB,7100,18.2
10,Tyreek,Hill,WR,KC,7700,19.4
11,Sammy,Watkins,WR,KC,4700,11.1
12,Antonio,Brown,WR,TB,5200,12.2
13,Mike,Evans,WR,TB,6600,15.8
14,Scotty,Miller,WR,TB,4200,8.8
15,Mecole,Hardman,WR,KC,4100,8.7
16,Travis,Kelce,TE,KC,8200,20.4
17,Rob,Gronkowski,TE,TB,4400,9.3
18,Cameron,Brate,TE,TB,4600,10.1
19,Kansas City,Defense,DST,KC,3300,5.2
20,Tampa,Defense,DST,TB,3100,5.1
""")

In [45]:
players = []
for row in csv.DictReader(data):
    players.append(Player(**row))

In [46]:
o.load_players(players)

In [47]:
o.players == players

True